In [ ]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

#

# Code:

In [ ]:
# Poplar 1x4". Cut
BlockHeight = 68.0  # [mm]
BlockLength = 143  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


In [ ]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

cnc.reset()

In [ ]:
if "Alarm" in cnc.status:
    cnc.home()

In [ ]:
cnc.init()

# Generate Program


In [ ]:
grid_spacing = 1 # [mm]

In [ ]:
Xs = np.arange(grid_spacing, BlockLength, grid_spacing)
Ys = np.arange(grid_spacing, BlockHeight, grid_spacing)

In [ ]:
Xs

In [ ]:
Ys

In [ ]:
len(Xs)

In [ ]:
Zs=np.array([-5, 0, 5])+-14
Zs

In [ ]:
n_powers = int(len(Xs)/len(Zs))
powers = np.linspace(50, 255, n_powers, dtype=np.uint8)
powers

In [ ]:
dwells = [0.01, 0.05, 0.10, 0.50, 1, 2, 2.5, 5]

In [ ]:
len(dwells)

In [ ]:
for idx3, dwell in enumerate(dwells):
    for idx1, Z in enumerate(Zs):
        for idx2, power in enumerate(powers):
            idx = (idx1*len(powers)+idx2)
            X=Xs[idx]
            Y=Ys[idx3]     

In [ ]:
def init(**kwargs):
    program = GCode.GCode(**kwargs)
    program.G0(F=500)
    program.G1(F=500)
    program.G21() # Metric Units
    program.G90() # Absolute positioning.
    return program

In [ ]:
def dot(X=10, Y=10, Z=0, power=1, dwell=1, **kwargs):
    program = init(**kwargs)
    
    program.G0(X=X, Y=Y, Z=Z)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.

    return program

In [ ]:
programs=list()
for idx3, dwell in enumerate(dwells):
    for idx1, Z in enumerate(Zs):
        for idx2, power in enumerate(powers):
            idx = (idx1*len(powers)+idx2)
            X=Xs[idx]
            Y=Ys[idx3]
            cfg = {
                "X": X,
                "Y": Y,
                "Z": Z,
                "power": power,
                "dwell": dwell,
                "machine": cnc
            }
            program = dot(**cfg)
            programs.append(program)

In [ ]:
programs

In [ ]:
import time

In [ ]:
for idx, program in enumerate(programs):
    print("{}: ".format(idx), end="")
    t1=time.time()
    out = cnc.run(program)
    t2=time.time()
    print("{}/{} {}".format(out, t2-t1, (out/t2-t1)))

In [ ]:
programs[0]